# SWAST Forecasting Tool

An ensemble of Regression with ARIMA Errors and Facebook Prophet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

check statsmodels version

In [ ]:
import statsmodels as sm
print(sm.__version__)

import os

os.getcwd()

In [ ]:
from swast_forecast.utility import (pre_process_daily_data, 
                                    default_ensemble,
                                    forecast, 
                                    multi_region_forecast)

## Constants

In [ ]:
PATH = '../ambo_data/Daily_Responses_5_Years_2019_full.csv'

## Read in the data

In [ ]:
clean = pre_process_daily_data(path=PATH, 
                               observation_col='Actual_Value', 
                               index_col='Actual_dt')
clean.head()

## Creating and fitting an Ensemble model to a region

The easy way to create an ensemble model is to call the `default_ensemble()` function form the utility module.  This returns the best known forecasting model.

In [ ]:
model = default_ensemble()
model

The code above informs us that the ensemble includes a Regression model with ARIMA errors with parameters (1, 1, 3)(1, 0, 1, 7).  By default a Prophet model will create a 80\% prediction interval 100(1-alpha)

To fit we call the `.fit()` method and pass in a `pd.Series` (or `pd.DataFrame`) that contains the historical observations.  By default you do not need to pass in holidays.  The ensemble will model new years day automatically (via Prophet's holidays function and as a dummy variable in the Regression with ARIMA errors).

In [ ]:
#example - fitting Wiltshire - this will take a few seconds.
model.fit(clean['Wiltshire'])

## Forecasting an individual region.

Use the `.predict()` method to make a forecast.  The method takes 3 parameters:

* **horizon**: int - the forecast horizon e.g. 84 days
* **alpha**: float, optional (default=0.05) - a value between 0 and 1 and used to construct a 100(1 - alpha) prediction interval. E.g. alpha=0.2 returns a 80\% interval.  
* **return_all_models**: bool, optional (default=False). If sets to true returns the ensemble prediction AND the Prophet and Regression predictions.

In [ ]:
#example 1: predict 5 days ahead - remember we have fitted Wiltshire training data.
forecast_frame = model.predict(horizon=5)
forecast_frame

The method returns a `pd.DataFrame` containing mean forecast (yhat) and an upper and lower prediction interval.  The code below demonstrates how to return predictions from both the ARIMA and Prophet models.  We will also return a different prediction interval.

In [ ]:
#example 2: predict 5 days ahead, return 95% PI and individual model preds
forecast_frame = model.predict(horizon=5, alpha=0.05, return_all_models=True)
forecast_frame

## An 'all in one' forecast function

As an alternative to the above the `utility` module contains a convenience function called `forecast`.  This is an all-in-one function.  Just pass in your training data (for a single time series) and horizon.

In [ ]:
forecast(clean['Wiltshire'], 
         horizon=6, 
         alpha=0.2,
         return_all_models=False)

## Forecasting multiple regions in one go.

If there are multiple regions to forecast put all of the training data into the same frame (see `clean`) and pass this to the `multi_region_forecast()` function from the `utility` module.

This is an efficient function as it runs the forecasts in parrallel across your CPU cores.  E.g. if you have a 4 cores then 4 regions will be forecast simultaneously.  This will reduce model run time (assuming you have more than one Core).

In [ ]:
#note depending on your machine this will take 20 seconds to run.
forecasts = multi_region_forecast(y_train=clean, horizon=8)

In [ ]:
#the function returns a list of pd.DataFrame's
type(forecasts)

In [ ]:
#results for BNSSG
forecasts[0]

In [ ]:
#results for Cornwall is at index 1 etc.
forecasts[1]